In [1]:
rdd = sc.textFile("../data/taxi-data-sorted-small.csv.bz2")
rdd = rdd.map(lambda x: x.split(','))

In [2]:
# import bz2
# with bz2.open("../data/taxi-data-sorted-small.csv.bz2", 'rt') as f:
#     content = f.readlines()

In [3]:
# rdd = sc.parallelize(content[:5000])

In [4]:
# rdd = rdd.map(lambda x: x.split(","))

In [2]:
import numpy as np

In [3]:
def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False

def correctRows(p):
    if(len(p) == 17):
        if(isfloat(p[4]) and isfloat(p[5]) and isfloat(p[11]) and isfloat(p[12]) and isfloat(p[16])):
            if(float(p[4]) != 0 and float(p[5]) != 0 and float(p[11]) != 0 and float(p[16])):
                return p

In [4]:
texilinesCorrect = rdd.filter(correctRows)

In [5]:
n = texilinesCorrect.count()

In [6]:
data = texilinesCorrect.map(lambda x: (np.array([1.0, float(x[4]), \
                                         float(x[5]), float(x[11]), \
                                         float(x[12]), float(x[16])])))

In [7]:
prev_w = prev_w = np.full(5, 0.1)
WtX = data.map(lambda x: (np.multiply(prev_w, x[:-1]), x))\
            .map(lambda x: (x[0].sum(), x[1]))

In [8]:
WtX.take(1)

[(12.544, array([  1.  , 120.  ,   0.44,   3.5 ,   0.5 ,   4.5 ]))]

In [9]:
errors_loss = WtX.map(lambda x: (np.subtract(x[0], x[1][5]), x[1]))\

In [10]:
errors.take(1)

[(8.044, array([  1.  , 120.  ,   0.44,   3.5 ,   0.5 ,   4.5 ]))]

In [11]:
loss = errors.map(lambda x:(1, x[0]**2))\
            .reduceByKey(lambda x1, x2: x1 + x2)

In [18]:
loss.collect()

[(1, 9319805873.655169)]

In [46]:
dW_loss = errors.map(lambda x: (1, (np.multiply(x[0], x[1][: -1]), x[0]**2)))\
       .reduceByKey(lambda x1, x2: (np.add(x1[0], x2[0]), x1[1] + x2[1]))

In [47]:
dW_loss.collect()

[(1, (array([1.10178961e+08, 1.13126232e+11, 5.24187237e+08, 1.91686238e+09,
          3.32014498e+07]), 9319805873.655169))]

In [45]:
dW_loss.take(1)

[(1,
  (array([  8.044  , 965.28   ,   3.53936,  28.154  ,   4.022  ]),
   64.70593600000001))]

In [32]:
a = np.array([1, 2,3])
b =np.array([1,2,3])
np.add(a, b)

array([2, 4, 6])

In [50]:
import sys

In [55]:
def gradientDescent(learningRate = 0.001, num_iteration = 100):
    prev_W= np.full(5, 0.1)
    count = 0
    # precision = 0.00000001
    # prev_stepsize = 1
    prev_loss = sys.float_info.max
    while(num_iteration > count):
        # Calculate linear combination w1 * x1 + w2 * x2.....
        # and keep original data in each row for later calculation.
        WtX = data.map(lambda x: (np.multiply(prev_w, x[:-1]), x))\
            .map(lambda x: (x[0].sum(), x[1]))
        
        # Calculate error which is target - prediction
        errors = WtX.map(lambda x: (np.subtract(x[0], x[1][5]), x[1]))
        
        
        # Calculate weight and loss
        dW_loss = errors.map(lambda x: (1, (np.multiply(x[0], x[1][: -1]), x[0]**2)))\
                .reduceByKey(lambda x1, x2: (np.add(x1[0], x2[0]), x1[1] + x2[1]))\
                .collect()
        
        # Extract weight from list
        dW = dW_loss[0][1][0]
        
         # Calculate current loss 
        cur_loss = dW_loss[0][1][1]
        
        # Set current weight 
        cur_W = np.subtract(prev_W, learningRate*dW)
        
       
        
        # Bold Driver logic
        if cur_loss < prev_loss:
            learningRate += 0.001
        else:
            learningRate -= 0.001
        
        print("Iteration: ", count)
        print("W: " ,cur_W)
        
        count += 1
        
        # Calculate stepsize
        # prev_stepsize = np.mean(np.abs(np.subtract(prev_W, cur_W)))
        
        prev_W = cur_W
        
        prev_loss = cur_loss
        
        
    return cur_W

In [56]:
W = gradientDescent()

Iteration:  0
W:  [-1.10178861e+05 -1.13126232e+08 -5.24187137e+05 -1.91686228e+06
 -3.32013498e+04]
Iteration:  1
W:  [-3.30536783e+05 -3.39378695e+08 -1.57256161e+06 -5.75058704e+06
 -9.96042493e+04]
Iteration:  2
W:  [-4.40715744e+05 -4.52504927e+08 -2.09674885e+06 -7.66744942e+06
 -1.32805699e+05]
Iteration:  3
W:  [-4.40715744e+05 -4.52504927e+08 -2.09674885e+06 -7.66744942e+06
 -1.32805699e+05]
Iteration:  4
W:  [-3.30536783e+05 -3.39378695e+08 -1.57256161e+06 -5.75058704e+06
 -9.96042493e+04]
Iteration:  5
W:  [-1.10178861e+05 -1.13126232e+08 -5.24187137e+05 -1.91686228e+06
 -3.32013498e+04]
Iteration:  6
W:  [2.20358022e+05 2.26252464e+08 1.04837457e+06 3.83372486e+06
 6.64029995e+04]
Iteration:  7
W:  [6.61073866e+05 6.78757390e+08 3.14512352e+06 1.15011744e+07
 1.99208799e+05]
Iteration:  8
W:  [1.21196867e+06 1.24438855e+09 5.76605971e+06 2.10854863e+07
 3.65216047e+05]
Iteration:  9
W:  [1.87304244e+06 1.92314594e+09 8.91118313e+06 3.25866606e+07
 5.64424746e+05]
Iteration:

Iteration:  85
W:  [3.74498289e+08 3.84516062e+11 1.78171242e+09 6.51541523e+09
 1.12851728e+08]
Iteration:  86
W:  [3.83643142e+08 3.93905539e+11 1.82521996e+09 6.67451481e+09
 1.15607448e+08]
Iteration:  87
W:  [3.92898175e+08 4.03408142e+11 1.86925169e+09 6.83553125e+09
 1.18396370e+08]
Iteration:  88
W:  [4.02263387e+08 4.13023872e+11 1.91380760e+09 6.99846455e+09
 1.21218493e+08]
Iteration:  89
W:  [4.11738777e+08 4.22752728e+11 1.95888770e+09 7.16331472e+09
 1.24073818e+08]
Iteration:  90
W:  [4.21324347e+08 4.32594710e+11 2.00449199e+09 7.33008175e+09
 1.26962344e+08]
Iteration:  91
W:  [4.31020096e+08 4.42549818e+11 2.05062047e+09 7.49876563e+09
 1.29884072e+08]
Iteration:  92
W:  [4.40826023e+08 4.52618053e+11 2.09727313e+09 7.66936639e+09
 1.32839001e+08]
Iteration:  93
W:  [4.50742130e+08 4.62799414e+11 2.14444999e+09 7.84188400e+09
 1.35827131e+08]
Iteration:  94
W:  [4.60768415e+08 4.73093901e+11 2.19215102e+09 8.01631848e+09
 1.38848463e+08]
Iteration:  95
W:  [4.70904879

In [51]:
# data = texilinesCorrect.flatMap(lambda x: [(1, np.array([1.0])), (4, np.array([float(x[4])])), \
#                                         (5, np.array([float(x[5])])), (11, np.array([float(x[11])])), \
#                                         (12, np.array([float(x[12])])), (16, np.array([float(x[16])]))])

In [53]:
# def appendToCol(x1, x2):
#     col_one = np.append(x1[0], x2[0])
#     col_idx4 = np.append(x1[1], x2[1])
#     col_idx5 = np.append(x1[2], x2[2])
#     col_idx11 = np.append(x1[3], x2[3])
#     col_idx12 = np.append(x1[4], x2[4])
#     col_idx16 = np.append(x1[5], x2[5])
#     return np.array([col_one, col_idx4, col_idx5, col_idx11, col_idx12, col_idx16])

In [63]:
# prev_w = np.full(5, 0.1)
# transposed = data.reduceByKey(lambda x1, x2: np.append(x1, x2))
# errors = transposed.map(lambda x: (x[0], np.multiply(0.1, x[1])))